In [1]:
import numpy as np
import pandas as pd
from skopt import gp_minimize, gbrt_minimize 
from skopt.plots import plot_convergence, plot_objective
from rl4greencrab.envs.green_crab_monthly_env import greenCrabMonthEnv
from rl4greencrab.envs.green_crab_monthly_env_norm import greenCrabMonthEnvNormalized
from rl4greencrab import evaluate_agent, multiConstAction, simulator, environment_simulation, plot_selected_sizes
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, FloatSlider
from stable_baselines3 import PPO, TD3

In [2]:
config = {
    "Tmax" : 200
}

In [3]:
env = greenCrabMonthEnv(config)

/opt/conda/lib/python3.12/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [4]:
agent = multiConstAction(env=env, action=np.array([0.0, 0, 0.0]))
constantData = environment_simulation(env, agent)
constantDataDF = pd.DataFrame(constantData)
constantDataDF.head()

,t,obs0,obs1,act0,act1,act2,rew,rep,crab_pop
0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,0.0,0.0,0.0,0.0,0.0,-0.000061,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,0.0,0.0,0.0,0.0,0.0,-0.000123,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,0.0,0.0,0.0,0.0,0.0,-0.000184,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,0.0,0.0,0.0,0.0,0.0,-0.000245,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [5]:
crab_pop_df = pd.DataFrame(constantDataDF[constantDataDF['rep']==0]['crab_pop'].tolist(), columns=[f'crab_pop_{i}' for i in range(len(constantDataDF['crab_pop'][0]))])

In [6]:
new_column_names = [i for i in range(len(crab_pop_df.columns))]
crab_pop_df.columns = new_column_names

In [7]:
def change_config(w_mort, nmort, growth_k):
    config = {
        "w_mort_scale" : w_mort,
        "nmortality" : nmort,
        "growth_k": growth_k,
        "Tmax": 200
    }
    env = greenCrabMonthEnv(config)
    agent = multiConstAction(env=env, action=np.array([0.0, 0, 0.0]))
    constantData = environment_simulation(env, agent)
    constantDataDF = pd.DataFrame(constantData)
    crab_pop_df = pd.DataFrame(constantDataDF[constantDataDF['rep']==0]['crab_pop'].tolist(), 
                               columns=[f'crab_pop_{i}' for i in range(len(constantDataDF['crab_pop'][0]))])
    new_column_names = [i for i in range(len(crab_pop_df.columns))]
    crab_pop_df.columns = new_column_names
    return crab_pop_df

In [8]:
# Interactive plot function
def plot_size_distribution(time_step, w_mort_scale, nmort, growth_k):
    crab_pop_df = change_config(w_mort_scale, nmort, growth_k)
    plt.figure(figsize=(8, 5))
    plt.bar(crab_pop_df.columns, crab_pop_df.loc[time_step], color='blue', alpha=0.7)
    plt.title(f'Crab Size Distribution at {time_step}')
    plt.xlabel('Crab Size')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

In [9]:
interact(plot_size_distribution, time_step=crab_pop_df.index, 
         w_mort_scale=FloatSlider(min=0, max=2000, step=50, value=600), 
         nmort = FloatSlider(min=0, max=1, step=0.01, value=0.03),
        growth_k = FloatSlider(min=0, max=1, step=0.01, value=0.7),);

interactive(children=(Dropdown(description='time_step', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…

### Size distribution with no internvention

In [75]:
config = {
        "w_mort_scale" : 600,
        "growth_k": 0.70,
    }
env = greenCrabMonthEnv(config)
eval_env = greenCrabMonthEnvNormalized(config)

/opt/conda/lib/python3.12/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [88]:
agent = multiConstAction(env=env, action=np.array([0.0, 0, 0.0]))
noInterventionData = environment_simulation(env, agent)
noInterventionDataDF = pd.DataFrame(constantData)

In [87]:
def pop_df_gen(dataDF):
    df = pd.DataFrame(dataDF[dataDF['rep']==0]['crab_pop'].tolist(), 
                      columns=[f'crab_pop_{i}' for i in range(len(dataDF['crab_pop'][0]))])
    df.columns = [i for i in range(len(df.columns))]
    return df

In [89]:
def plot_size_distribution(dataDF, time_step):
    crab_pop_df = pop_df_gen(dataDF)
    plt.figure(figsize=(8, 5))
    plt.bar(crab_pop_df.columns, crab_pop_df.loc[time_step], color='blue', alpha=0.7)
    plt.title(f'Crab Size Distribution at {time_step}')
    plt.xlabel('Crab Size')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

In [107]:
def interact_size_plot(dataDF):
    return interact(
        lambda time_step: plot_size_distribution(dataDF, time_step),
         time_step=dataDF.index
    );

In [91]:
interact_size_plot(noInterventionDataDF)

interactive(children=(Dropdown(description='time_step', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…

<function __main__.interact_size_plot.<locals>.<lambda>(time_step)>

In [92]:
agent = multiConstAction(env=env, action=np.array([83.87232800633504, 596.3225575635984, 14.882297944474463]))
constantData = environment_simulation(env, agent)
constantDataDF = pd.DataFrame(constantData)

In [93]:
interact_size_plot(constantDataDF)

interactive(children=(Dropdown(description='time_step', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…

<function __main__.interact_size_plot.<locals>.<lambda>(time_step)>

In [96]:
ppoAgent = PPO.load("../saved_agents/PPO_gcmenorm", device="cpu")
td3Agent = TD3.load("../saved_agents/TD3_gcmenorm_curriculumn", device="cpu")

In [97]:
ppoDataDF = pd.DataFrame(environment_simulation(eval_env, ppoAgent))

In [98]:
interact_size_plot(ppoDataDF)

interactive(children=(Dropdown(description='time_step', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…

<function __main__.interact_size_plot.<locals>.<lambda>(time_step)>

In [131]:
dataDF_list = [noInterventionDataDF, constantDataDF, ppoDataDF]

In [138]:
def plot_size_distr_overlap(dataDF_list, time_step):
    plt.figure(figsize=(8, 5))
    for i, df in enumerate(dataDF_list):
        crab_pop_df = pop_df_gen(df)
        plt.bar(
            crab_pop_df.columns,
            crab_pop_df.loc[time_step],
            alpha=0.5,
            label=f"DataFrame {i+1}"
        )
    plt.legend()
    plt.title(f'Crab Size Distribution at {time_step}')
    plt.xlabel('Crab Size')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

def interact_size_plot(dataDF_list):
    return interact(
        lambda time_step: plot_size_distr_overlap(dataDF_list, time_step),
         time_step=pd.DataFrame(dataDF_list[0]).index
    );

In [139]:
interact_size_plot(dataDF_list)

interactive(children=(Dropdown(description='time_step', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…

<function __main__.interact_size_plot.<locals>.<lambda>(time_step)>